# Exercise 12: Cross validation
-----

In this exercise, we'll practice implementing cross validation techniques, including leave-one-out and k-fold cross validation. We'll use the `PimaIndiansDiabetes2` practice dataset, which has medical data on a group of Pima Native American women, including whether or not they have diabetes. This dataset is part of the `mlbench` package. We'll be using each person's medical history to predict whether or not they have been diagnosed with diabetes.

1. Data 1/1
2. Leave-one-out Cross Validation 4/4
3. Compare to cv.glm 3/3
4. Adjusting K and Reflection 2/2

# 1: Data (1 pts)
---

Load the `tidyverse`, `boot`, and `mlbench` packages (you may need to install `boot` and `mlbench`).

Load the `PimaIndiansDiabetes2` dataset using the `data()` function. Drop the `insulin` column (it just has a lot of missing data) and then drop `NA`s from the rest of the dataset. Save your updated dataset to a new variable name. Finally, print the dimensions of your new dataset, and look at the first few lines of data.

In [ ]:
library(tidyverse)
install.packages("mlbench")
install.packages("boot")
library(mlbench)
library(boot)
data(PimaIndiansDiabetes2)
df_diab <- PimaIndiansDiabetes2
df_diab$insulin <- NULL
df_diab <- na.omit(df_diab)
print(dim(df_diab))
head(df_diab)



── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.5.1     ✔ tibble    3.2.1
✔ lubridate 1.9.4     ✔ tidyr     1.3.1
✔ purrr     1.0.4     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors
Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



[1] 532   8


,pregnant,glucose,pressure,triceps,mass,pedigree,age,diabetes
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>
1,6,148,72,35,33.6,0.627,50,pos
2,1,85,66,29,26.6,0.351,31,neg
4,1,89,66,23,28.1,0.167,21,neg
5,0,137,40,35,43.1,2.288,33,pos
7,3,78,50,32,31.0,0.248,26,pos
9,2,197,70,45,30.5,0.158,53,pos


(Note that in medical contexts, `pedigree` refers to a system of measuring family history of a condition. So here, higher numbers mean greater family history of diabetes. You can read more about this dataset [here](https://rdrr.io/cran/mlbench/man/PimaIndiansDiabetes.html).)

# 2. Leave-one-out Cross Validation (4 pts)

In the tutorial, we learned how to fit leave-one-out cross validation using the `cv.glm` function from the `boot` package. But we can also do this manually using `predict()` like we have in the past.

Let's predict `diabetes`, a dichotomous outcome, using all the other variables in our modified dataset.

First, fit a logistic regression model using all of the observations except the very first one. Then use your fitted model to predict whether your holdout case is positive or negative for diabetes. Remember that logistic regression coefficients are in **log-odds**, meaning that if an output is positive, the probability of the outcome is greater than 50%; if the output is negative, the probability of the outcome is less than 50%.

Compare your result to the actual response in row one above. Did your model correctly classify this observation?

In [ ]:
holdout <- df_diab[1, ]
train <- df_diab[-1, ]
model_log <- glm(diabetes ~ pregnant + glucose + pressure + triceps + mass + pedigree + age, data = train, family = binomial (link = "logit"))
hold_pred <- predict(model_log, newdata = holdout, type = "response")
predicted<- ifelse(hold_pred > 0.5, "pos", "neg")
actual <- as.character(holdout$diabetes)
print(paste("Predicted class:", predicted))
print(paste("Actual class:", actual))



[1] "Predicted class: pos"
[1] "Actual class: pos"


So we just calculated a single iteration of LOOCV. We used 531 rows of our data to fit a model to predict the outcome of the last row.

Below, use a `for` loop to iterate through the rest of your dataset doing the same thing. You will need to:
* Create a data frame `results` with two columns: one named `actual` which holds the true classification for each observation, and one named `predicted`, which should be filled with `NA`s. This is where you'll store the output of your loop.
* Create a loop that runs through each row of your data, pulls that observation out, trains your model on the remaining data, and then tests the fitted model on your test observation.
* Store your model *predictions* ("pos" or "neg" -- not the log-odds) in the `predicted` column of your `results` dataframe

After you run your loop, print the first few lines of `results`.

In [ ]:
results <- data.frame(actual= df_diab$diabetes, predicted = rep(NA, nrow(df_diab)))

for (i in 1:nrow(df_diab)){
holdout1 <- df_diab[i, ]
train1 <- df_diab [-i, ]

model1 <- glm(diabetes ~ pregnant + glucose + pressure + triceps + mass + pedigree + age,
              data = train1, family = binomial (link = "logit"))
hold_pred1 <- predict(model1, newdata = holdout1, type = "response")
predicted_class<- ifelse(hold_pred1 > 0.5, "pos", "neg")
results$predicted[i]<- predicted_class

}

head(results)

,actual,predicted
,<fct>,<chr>
1,pos,pos
2,neg,neg
3,neg,neg
4,pos,pos
5,pos,neg
6,pos,pos


Now, calculate the overall error of your model. What proportion of cases were incorrectly classified?

In [ ]:
incorrect_class <- sum (results$actual != results$predicted)
total_class <- nrow(results)
error_rate <- incorrect_class/total_class
print(paste("error rate:", error_rate))


[1] "error rate: 0.221804511278195"


# 3. Compare to `cv.glm` (3 pts)

Now, let's compare this result to the `cv.glm` function. Using the tutorial as a guide, use `cv.glm` to run LOOCV on the data, using the same model (i.e., still using all of the variables to predict diabetes diagnosis).

Note that, because this is a `classification` problem and not a regression problem like in the tutorial, we need to adjust the `cost` argument of `cv.glm`. We can read more about this in the docs:

In [ ]:
library(boot)

model2 <- glm(diabetes ~ pregnant + glucose + pressure + triceps + mass + pedigree + age,
              data = df_diab, family = binomial (link = "logit"))

cost_f <- function(r, pi = 0.5) {
    mean(abs(r-(pi> 0.5)))
}

cv_glm <- cv.glm(df_diab, model2, cost = cost_f )
print(paste(" Error rate cv.glm", cv_glm$delta[1]))


Here, we see `cost` is defined as:
> "A function of two vector arguments specifying the cost function for the cross-validation. The first argument to cost should correspond to the **observed responses** and the second argument should correspond to the **predicted or fitted responses** from the generalized linear model."

In the example code (scroll to bottom of the docs), we see that the appropriate cost function for a binary classification is

``
cost <- function(r, pi = 0) mean(abs(r-pi) > 0.5)
``

Where `r` is the vector of observed responses (technically "pos" and "neg", but R treats these as 1 and 0 under the hood), and `pi` is the vector of *probabilities* (not log-odds) fit by the model. Thus, this boils down to our error: what proportion of observations were incorrectly classified. You will need to include this code below.

In [ ]:
cost_f <- function(r, pi = 0.5) {
    mean(abs(r-(pi> 0.5)))
}

cv_glm <- cv.glm(df_diab, model2, cost = cost_f )
print(paste(" Error rate cv.glm", cv_glm$delta[1]))


[1] " Error rate cv.glm 0.221804511278196"


How do your results compare to your manual LOOCV above?

> The error rate between the manual and the cv.glm model are identical. The only difference is the manual LOOCV takes more time to do the same process.




# 4. Adjusting K and Reflection (2 pts)

Recall that LOOCV has some drawbacks. In particular, it has quite high *variance* which can lead to poor performance on new test data. We can reduce this variance by increasing K.

Below, re-run your cross validation using `cv.glm` with `k` set to 3, 5, 10, and 15.

In [ ]:
set.seed(1)

cost_f <- function(r, pi = 0.5) {
    mean(abs(r-(pi> 0.5)))
}
model2 <- glm(diabetes ~ pregnant + glucose + pressure + triceps + mass + pedigree + age,
              data = df_diab, family = binomial (link = "logit"))
k_values <- c(3, 5, 10, 15)

 for (k in k_values) {
    cv_k <- cv.glm (df_diab, model2, cost = cost_f, K= k)
    print(paste("K =", k,", Error rate", cv_k$delta[1]))
}

[1] "K = 3 , Error rate 0.210526315789474"
[1] "K = 5 , Error rate 0.216165413533835"
[1] "K = 10 , Error rate 0.223684210526316"
[1] "K = 15 , Error rate 0.227443609022556"


#### Reflection

How do your errors compare to your LOOCV error above? How do they change as k increases?
> * We can observe that K= 3 and K= 5 have somewhat lower error rates than our LOOCV output. In K=10 we have similar performance and K=15 is higher than our previous value.
> The model seems to be adequate because the variation is small when using a variety of cross-validation methods.


If you change the random seed above, you'll get slightly different errors. If you were to do the same with your LOOCV above , would you expect to get different results each time? Why or why not?
> No, LOOCV will get the same outcome despite of the random seed. This is because there is no random shuffling like in the k-fold cross-validation.



**DUE:** 5pm March 24, 2025

**IMPORTANT** Did you collaborate with anyone on this assignment? If so, list their names here.
> *Someone's Name*
>
>
